# **Style Transfer**

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers
!wget https://raw.githubusercontent.com/smilegate-ai/korean_smile_style_dataset/main/smilestyle_dataset.tsv

--2023-10-12 13:43:59--  https://raw.githubusercontent.com/smilegate-ai/korean_smile_style_dataset/main/smilestyle_dataset.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2357401 (2.2M) [text/plain]
Saving to: ‘smilestyle_dataset.tsv.2’

smilestyle_dataset. 100%[===================>]   2.25M  --.-KB/s    in 0.05s   

2023-10-12 13:43:59 (43.0 MB/s) - ‘smilestyle_dataset.tsv.2’ saved [2357401/2357401]



In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from IPython.display import display
from typing import Dict

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/dab/순화 파일 (1).xlsx')
df.head()

,Unnamed: 0,문장,순화문장,"단순 욕설 1, 순화가능 0"
0,1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0
1,2,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,0
2,4,나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임,나이먹고 피시방가는 사람들은 머리에 똥만찬 사람들임,0
3,5,출산청같은 소리하네. 우리나라 계집들 다 의무적으로 군대보내면 해결될 일이다. 정신...,출산청같은 소리하네. 우리나라 여자들 다 의무적으로 군대보내면 해결될 일이다. 정신...,0
4,6,가짜 남자는 어떻게 생겼냐?,가짜 남자는 어떻게 생겼냐?,0


In [ ]:
df = df.drop(['Unnamed: 0', '단순 욕설 1, 순화가능 0'], axis=1)
df.head()

,문장,순화문장
0,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...
1,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...
2,나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임,나이먹고 피시방가는 사람들은 머리에 똥만찬 사람들임
3,출산청같은 소리하네. 우리나라 계집들 다 의무적으로 군대보내면 해결될 일이다. 정신...,출산청같은 소리하네. 우리나라 여자들 다 의무적으로 군대보내면 해결될 일이다. 정신...
4,가짜 남자는 어떻게 생겼냐?,가짜 남자는 어떻게 생겼냐?


In [ ]:
print(df.tail())
print(len(df))

                                     문장                               순화문장
4633                      개슬람 개독들은 광신도들                     이슬람 개신교들은 광신도들
4634             뭐든 보여주기식으로 할려니깐 문제인거야.             뭐든 보여주기식으로 할려니깐 문제인거야.
4635  정확한 근거도 없이 그런일 없다는 니네가 더 문제다. 경향.  정확한 근거도 없이 그런일 없다는 니네가 더 문제다. 경향.
4636                   죽고싶어하던데...이중인격자니                   죽고싶어하던데...이중인격자니
4637        저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ        저게 시대적언어면 한남충도 시대적언어 아니냐 ㅋㅋ
4638


In [ ]:
model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


## Dataset class 만들기

In [ ]:
style_map = {'문장' : '원래문장', '순화문장' : '순화문장'}

학습에 필요한 Dataset을 만든다, 여러 스타일을 가진 한 문장을 가져온 뒤, 두가지 스타일을 임의로 추출해서 하나를 원본 문장으로 지정하고 encoder의 입력으로 사용하고, 다른 하나를 목표 문장으로 decoder의 입력으로 사용한다.

In [ ]:

class TextStyleTransferDataset(Dataset):
  def __init__(self,
               df: pd.DataFrame,
               tokenizer: Tokenizer
               ):
    self.df = df
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    row = self.df.iloc[index, :].dropna().sample(2, replace=True)
    text1 = row[0]
    text2 = row[1]
    target_style = row.index[1]
    target_style_name = style_map[target_style]

    encoder_text = f"{target_style_name} 말투로 변환:{text1}"
    decoder_text = f"{text2}{self.tokenizer.eos_token}"
    model_inputs = self.tokenizer(encoder_text, max_length=64, truncation=True)

    with self.tokenizer.as_target_tokenizer():
      labels = tokenizer(decoder_text, max_length=64, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    del model_inputs['token_type_ids']

    return model_inputs

In [ ]:
dataset = TextStyleTransferDataset(df, tokenizer)
out = dataset[0]
print(out['input_ids'])
print(out['labels'])
print(tokenizer.decode(out['input_ids']))
print(tokenizer.decode(out['labels']))

out = dataset[1]
print(out['input_ids'])
print(out['labels'])
print(tokenizer.decode(out['input_ids']))
print(tokenizer.decode(out['labels']))

[14408, 13714, 10671, 12074, 14070, 13282, 10338, 14296, 13716, 257, 24620, 11280, 17816, 11863, 14983, 9866, 10830, 9698, 14557, 10769, 15837, 18619, 25875, 16752, 15897, 14130, 14113, 11444, 10486, 15417, 13417, 14356, 12865, 16247, 14239, 14787, 14129, 14943, 280, 28844, 20894, 15532, 12005, 22093, 236, 27043]
[17733, 11280, 17816, 11863, 14983, 9866, 10830, 9698, 14557, 10769, 15837, 18619, 25875, 16752, 15897, 14130, 14113, 11444, 10486, 15417, 13417, 14356, 12865, 16247, 14239, 14787, 14129, 14943, 280, 28844, 20894, 15532, 12005, 22093, 236, 27043, 1]
순화문장 말투로 변환:아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.
아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. 10 16까지 IQ 떨어지고 출산 위험은 400% 다.</s>
[14408, 13714, 10671, 12074, 14070, 13282, 10338, 14296, 13716, 257, 11696, 9622, 14851, 15279, 1700, 9785, 14167, 14056, 14083, 15105, 24598, 9581, 14218, 15165, 14135, 10325, 9754, 232, 15439, 14167, 14166, 27256, 14376, 9754, 232, 14747, 18808, 10338, 28097, 953

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split

# 학습을 위해 train, test set으로 나눈다.
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
print(len(df_train), len(df_test))

4174 464


In [ ]:
train_dataset = TextStyleTransferDataset(
    df_train,
    tokenizer
)
test_dataset = TextStyleTransferDataset(
    df_test,
    tokenizer
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
model_path = "/content/drive/MyDrive/dab/2nd bart/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=24, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.723000,0.436368
1000,0.373300,0.420839
1500,0.280800,0.423929
2000,0.227800,0.450752
2500,0.189000,0.422533
3000,0.160100,0.445982
3500,0.135000,0.435630
4000,0.109900,0.372235
4500,0.089500,0.411500
5000,0.077300,0.448422


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

TrainOutput(global_step=6264, training_loss=0.20078810345319648, metrics={'train_runtime': 1792.9598, 'train_samples_per_second': 55.872, 'train_steps_per_second': 3.494, 'total_flos': 3659049921945600.0, 'train_loss': 0.20078810345319648, 'epoch': 24.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/dab/2nd bart/")

## Pipeline을 이용해서 학습한 모델로 텍스트 생성해보기

In [ ]:
from transformers import pipeline

nlg_pipeline = pipeline('text2text-generation',model=model_path, tokenizer=model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
def generate_text(pipe, text, target_style, num_return_sequences=5, max_length=60):
  target_style_name = style_map[target_style]
  text = f"{target_style_name} 말투로 변환:{text}"
  out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
  return [x['generated_text'] for x in out]

In [ ]:
src_text = """
우리공장 인도네시아 개슬람새끼4월달 졸라 바쁠때 결근했음다음날 왜 무단결근 했냐고 물어보니간 머리가 아퍼서 이슬람사원에 기도하러 갔어요...ㅅㅂ
"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=1000)[0])

입력 문장: 
우리공장 인도네시아 개슬람새끼4월달 졸라 바쁠때 결근했음다음날 왜 무단결근 했냐고 물어보니간 머리가 아퍼서 이슬람사원에 기도하러 갔어요...ㅅㅂ

순화 문장:
 우리공장 인도네시아 이슬람사람4월달 엄청 바쁠때 결근했음 다음날 왜 무단결근 했냐고 물어보니간 머리가 아퍼서 이슬람사원에 기도하러 갔어요...


In [ ]:
src_text = """
아놔 씨발쌍도새끼들은 왜 지네들 까면 전라도 라는거냐? 이미친새끼들아 너네들 존네 허세부려서 존나 싫다고 개새끼들아 말귀를 알아 쳐먹으라고 . 부산제외 부산은 좋다ㅋ
"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=1000)[0])

입력 문장: 
아놔 씨발쌍도새끼들은 왜 지네들 까면 전라도 라는거냐? 이미친새끼들아 너네들 존네 허세부려서 존나 싫다고 개새끼들아 말귀를 알아 쳐먹으라고 . 부산제외 부산은 좋다ㅋ

순화 문장:
 전라도 사람들은 왜 지네들 까면 전라도 라는거냐? 너네들 엄청 허세부려서 매우 싫다고 부산제외 부산은 좋다ᄏ


In [ ]:
src_text = """
좃라도는 삼국시대까지 마한이라는 외국이었다는거 , 고려 현종때 1018년이 되어서야 " 좃라도는 우리한민족으로 편입되었음"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=1000)[0])

입력 문장: 
좃라도는 삼국시대까지 마한이라는 외국이었다는거 , 고려 현종때 1018년이 되어서야 " 좃라도는 우리한민족으로 편입되었음
순화 문장:
 전라도는 삼국시대까지 마한이라는 외국이었다는거 , 고려 현종때 1018년이 되어서야 " 전라도는 우리한민족으로 편입되었음


In [ ]:
src_text = """
틀딱은 너그럽고 인자하고 연륜있게 나오고 주인공은 까칠하게 나옴 개좆같은영화"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=1000)[0])

입력 문장: 
틀딱은 너그럽고 인자하고 연륜있게 나오고 주인공은 까칠하게 나옴 개좆같은영화
순화 문장:
 노인은 너그럽고 인자하고 연륜있게 나오고 주인공은 까칠하게 나옴 별로인 영화


train dataset에 없는 문장 test

In [ ]:
src_text = """
그 전두엽 녹아내린 좀비 새끼는 저녁에 어떤 술 쳐먹을까 고민하고 있다"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=1000)[0])

입력 문장: 
그 전두엽 녹아내린 좀비 새끼는 저녁에 어떤 술 쳐먹을까 고민하고 있다
순화 문장:
 그 전두엽 녹아내린 좀비는 저녁에 어떤 술 먹을까 고민하고 있다


In [ ]:
src_text = """
아니씨발 진짜 조오오오온나 답답하다 세월아 내월아 벌써 2년 다되어가는데 아무도 구속 안되고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 씨발진짜 내년 총선 개발리고 윤석열 탄핵되서
김건희랑 손잡고 감옥가라 시발 진심이다 병신같은 국힘 검찰 ㅉㅉ"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=2000)[0])

입력 문장: 
아니씨발 진짜 조오오오온나 답답하다 세월아 내월아 벌써 2년 다되어가는데 아무도 구속 안되고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 씨발진짜 내년 총선 개발리고 윤석열 탄핵되서
김건희랑 손잡고 감옥가라 시발 진심이다 병신같은 국힘 검찰 ㅉㅉ
순화 문장:
 아니 진짜오오오오온나 답답하다 세월아 내월아 벌써 2년 다되어가는데 아무도 구속 안되고 ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ 내년 총선 개발리고 윤석열 탄핵되서


In [ ]:
src_text = """
ㅆㅂ 원숭이 만들어서 서울 조진게 누군데"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=2000)[0])

입력 문장: 
ㅆㅂ 원숭이 만들어서 서울 조진게 누군데
순화 문장:
 원숭이 만들어서 서울 조진게 누군데


In [ ]:
src_text = """

"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=2000)[0])

In [ ]:
src_text = """
보수라고 한 적 없는데 이 새끼처럼 좌좀새끼들이 좌파 윤석렬을 왜 미워하는지 미스테리함 ㅋㅋ
"""
print("입력 문장:", src_text)
print("순화 문장:\n", generate_text(nlg_pipeline, src_text, '순화문장', num_return_sequences=1, max_length=2000)[0])


입력 문장: 
보수라고 한 적 없는데 이 새끼처럼 좌좀새끼들이 좌파 윤석렬을 왜 미워하는지 미스테리함 ㅋㅋ

순화 문장:
 보수라고 한 적 없는데 이 사람처럼 좌파 윤석렬을 왜 미워하는지 미스테리함 ᄏᄏ



# **단어쌍 이용**

In [ ]:
pair_vocab = pd.read_excel('/content/drive/MyDrive/dab/pair_vocab (1).xlsx')
pair_vocab.head()

,교정전,교정후
0,하노,하네
1,똥꼬충,게이
2,냄져,남자
3,이기,NaN
4,빠개다,웃다


In [ ]:
pair_vocab.tail()

,교정전,교정후
396,ㅂㅅㅌㅊ,바보같은
397,미친,이상한
398,친1년놈들,NaN
399,보혐,여혐
400,튀기,혼혈


In [ ]:
pair_vocab['교정후'] = pair_vocab['교정후'].fillna('')

In [ ]:
pair_vocab.head()

,교정전,교정후
0,하노,하네
1,똥꼬충,게이
2,냄져,남자
3,이기,
4,빠개다,웃다


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def preprocess_data(text):
    return tokenizer(text, return_tensors='pt', padding=True, truncation=True)

In [ ]:
pair_vocab['교정전'][0]

'하노'

In [ ]:
preprocess_data(pair_vocab['교정전'][0])

{'input_ids': tensor([[3, 2, 1]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:
pair_vocab['input']=None
pair_vocab['output']=None
pair_vocab.columns

Index(['교정전', '교정후', 'input', 'output'], dtype='object')

In [ ]:
for i in range(len(pair_vocab)):
  pair_vocab['input'][i] = preprocess_data(pair_vocab['교정전'][i])
  pair_vocab['output'][i] = preprocess_data(pair_vocab['교정후'][i])

ValueError: ignored